Lab 10


In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
import pathlib
import random
import keras
import keras.models as M
import keras.layers as L
import keras.backend as K

from google.colab import files, drive
from sklearn.model_selection import train_test_split


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
csvs =  sorted(str(p) for p in pathlib.Path('/content/drive/MyDrive/FontsPZNM/fonts').glob("*.csv"))

data = pd.DataFrame()
for csv in csvs[::2]:
  data = data.append(pd.read_csv(csv))

display(data.iloc[random.sample(range(0, len(data.index)), 10)])

<ipython-input-6-3f3ac343d04f>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(pd.read_csv(csv))
<ipython-input-6-3f3ac343d04f>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(pd.read_csv(csv))
<ipython-input-6-3f3ac343d04f>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(pd.read_csv(csv))
<ipython-input-6-3f3ac343d04f>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(pd.read_csv(csv))
<ipython-input-6-3f3ac343d04f>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = 

,font,fontVariant,m_label,strength,italic,orientation,m_top,m_left,originalH,originalW,...,r19c10,r19c11,r19c12,r19c13,r19c14,r19c15,r19c16,r19c17,r19c18,r19c19
9856,NUMERICS,scanned,48,0.4,0,0.0,0,0,37,23,...,196,160,118,6,1,1,1,1,1,1
61056,HANDPRINT,scanned,56,0.4,0,0.0,0,0,20,16,...,1,1,1,1,1,1,1,1,1,1
66723,HANDPRINT,scanned,53,0.4,0,0.0,0,0,20,20,...,48,1,1,1,1,1,1,1,1,1
1271,TXT,TXT,69,0.7,1,0.0,29,27,49,51,...,209,209,209,152,1,1,1,1,1,1
809,IMPRINT,IMPRINT MT SHADOW,213,0.7,1,0.0,20,29,63,53,...,139,69,6,1,1,1,1,1,1,1
743,GADUGI,GADUGI,5124,0.4,0,0.0,23,22,60,47,...,255,255,255,255,255,255,255,255,255,183
363,NINA,NINA,274,0.4,0,0.0,26,24,61,27,...,255,255,255,255,255,255,255,255,255,255
58245,HANDPRINT,scanned,53,0.4,0,0.0,0,0,18,20,...,1,1,1,1,1,1,1,1,1,1
4749,CALIBRI,CALIBRI LIGHT,48,0.4,1,0.0,42,23,42,31,...,173,87,11,1,1,1,1,1,1,1
48978,HANDPRINT,scanned,49,0.4,0,0.0,0,0,20,7,...,141,142,143,143,106,91,91,34,24,24


In [7]:
factorized = pd.factorize(data['font'])
data['font'] = factorized[0]
unique_cats_count = len(factorized[1])

print(f"Unique categories: {unique_cats_count}")

display(data.iloc[random.sample(range(0, len(data.index)), 10)])

Unique categories: 77


,font,fontVariant,m_label,strength,italic,orientation,m_top,m_left,originalH,originalW,...,r19c10,r19c11,r19c12,r19c13,r19c14,r19c15,r19c16,r19c17,r19c18,r19c19
13314,42,LUCIDA CALLIGRAPHY,352,0.4,0,0.0,21,24,65,40,...,138,99,20,1,1,1,1,1,1,1
5087,38,ISOCT,125,0.7,1,0.0,51,39,60,26,...,1,1,1,1,1,1,1,1,1,1
93,36,MICROSOFT HIMALAYA,4014,0.4,0,0.0,38,25,49,25,...,1,1,1,1,1,1,1,1,154,255
70723,51,scanned,47,0.4,0,0.0,0,0,14,8,...,1,1,1,1,1,1,1,1,1,1
65532,34,scanned,54,0.4,0,0.0,0,0,20,12,...,244,202,118,44,7,1,1,1,1,1
4376,27,FRANKLIN GOTHIC MEDIUM,269,0.7,0,0.0,35,22,50,30,...,255,255,255,255,154,86,35,1,1,1
2000,68,SYLFAEN,952,0.7,0,0.0,42,22,47,31,...,255,255,175,147,53,1,1,1,1,1
52358,34,scanned,49,0.4,0,0.0,0,0,20,11,...,1,1,1,1,1,1,1,1,1,1
1204,25,EUROROMAN,231,0.7,0,0.0,21,21,58,50,...,1,1,1,71,224,255,255,255,240,176
1698,69,MICROSOFT NEW TAI LUE,6546,0.4,1,0.0,47,27,36,44,...,255,255,255,140,11,1,1,1,1,1


In [8]:
train, test = train_test_split(data, test_size=0.15)
ignored_columns_for_train = ["font", "fontVariant", "m_label", "strength", "italic", "orientation", "m_top", "m_left", "originalH", "originalW", "h", "w"]

train_y = np.array(train["font"])
train_x = train.drop(columns=ignored_columns_for_train)

test_y = np.array(test["font"])
test_x = test.drop(columns=ignored_columns_for_train)
train_y_oh = keras.utils.to_categorical(train_y, unique_cats_count)
test_y_oh = keras.utils.to_categorical(test_y, unique_cats_count)
train_x = train_x.to_numpy()
test_x = test_x.to_numpy()
print(train_x)

[[  1   1  15 ...   1   1   1]
 [  1   1 105 ...   1   1   1]
 [  1   1   1 ...   1   1   1]
 ...
 [  1   1   1 ...   1   1   1]
 [  1   1   1 ...   1   1   1]
 [  1   1   1 ...   1   1   1]]


In [9]:
train_x = train_x.reshape((len(train_x), 20, 20, 1))
train_x = train_x.astype('float32') / 255

test_x = test_x.reshape((len(test_x), 20, 20, 1))
test_x = test_x.astype('float32') / 255
K.clear_session()

model = M.Sequential()
model.add(L.Conv2D(20, (3,3), activation='relu', input_shape= (20, 20, 1)))
model.add(L.Conv2D(20, (2,2), activation='relu'))
model.add(L.BatchNormalization())
model.add(L.Conv2D(40, (2, 2), activation='relu'))
model.add(L.Flatten())
model.add(L.Dense(unique_cats_count, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
results = model.fit(train_x, train_y_oh, batch_size=64, epochs=15, validation_data=(test_x, test_y_oh))

Epoch 1/15
5335/5335 [==============================] - 315s 59ms/step - loss: 1.5058 - accuracy: 0.6291 - val_loss: 1.2599 - val_accuracy: 0.6834
Epoch 2/15
5335/5335 [==============================] - 326s 61ms/step - loss: 1.0091 - accuracy: 0.7331 - val_loss: 1.1303 - val_accuracy: 0.7122
Epoch 3/15
5335/5335 [==============================] - 318s 60ms/step - loss: 0.8034 - accuracy: 0.7784 - val_loss: 1.1047 - val_accuracy: 0.7191
Epoch 4/15
5335/5335 [==============================] - 315s 59ms/step - loss: 0.6835 - accuracy: 0.8070 - val_loss: 1.1227 - val_accuracy: 0.7273
Epoch 5/15
5335/5335 [==============================] - 317s 59ms/step - loss: 0.6028 - accuracy: 0.8265 - val_loss: 1.1153 - val_accuracy: 0.7357
Epoch 6/15
5335/5335 [==============================] - 314s 59ms/step - loss: 0.5472 - accuracy: 0.8416 - val_loss: 1.1729 - val_accuracy: 0.7402
Epoch 7/15
5335/5335 [==============================] - 312s 59ms/step - loss: 0.5057 - accuracy: 0.8525 - val_loss: 1

In [10]:
print(f"Accuracy: {np.max(results.history['accuracy'])}")

Accuracy: 0.8951634168624878
